In [3]:
import pickle
import numpy as np
import pandas as pd
# from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix


In [4]:
train_df = pd.read_csv(r'C:\Users\ILYA\Downloads\train_sessions.csv\train_sessions.csv', index_col='session_id')
test_df = pd.read_csv(r'C:\Users\ILYA\Downloads\test_sessions.csv\test_sessions.csv', index_col='session_id')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ILYA\\Downloads\\train_sessions.csv\\train_sessions.csv'

In [3]:
train_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253557,3474,2013-11-25 10:26:54,3474.0,2013-11-25 10:26:58,141.0,2013-11-25 10:27:03,2428.0,2013-11-25 10:27:04,106.0,2013-11-25 10:27:13,...,2013-11-25 10:27:16,2428.0,2013-11-25 10:27:28,2428.0,2013-11-25 10:27:40,2428.0,2013-11-25 10:27:52,148.0,2013-11-25 10:27:53,0
253558,12727,2013-03-12 16:01:15,12727.0,2013-03-12 16:01:16,2215.0,2013-03-12 16:01:16,38.0,2013-03-12 16:01:17,2215.0,2013-03-12 16:01:17,...,2013-03-12 16:01:17,25444.0,2013-03-12 16:01:18,2215.0,2013-03-12 16:01:18,23.0,2013-03-12 16:01:18,21.0,2013-03-12 16:01:18,0
253559,2661,2013-09-12 14:05:03,15004.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:06:29,5562.0,2013-09-12 14:06:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
#Отбираем у данных для Х только сайты, без времени
site = ['site%s' % i for i in range(1,11)]

Xtr = train_df[site]

Xte = test_df[site]


#Замена пустых значений нулями 
Xtr = Xtr.fillna(0)
Xte = Xte.fillna(0)

y = train_df.target

In [42]:
#Создание новой фичи  по первому посещению Time1 смотрим и масштабируем целиком время первого посещения 
#for train
day1tr = pd.DataFrame({'day1': train_df['time1']})
day1tr.day1 = day1tr.day1.apply(lambda x: x[:7].replace('-', '0')).astype(int).apply(lambda x: x/2.013710e+06)

In [40]:
#for test
day1te = pd.DataFrame({'day1': test_df['time1']})
day1te.day1 = day1te.day1.apply(lambda x: x[:7].replace('-', '0')).astype(int).apply(lambda x: x/2.013710e+06)


In [69]:
#Cоздание единого датафрейма по фиче Day1 для fullds 
fullday1 = pd.concat([day1tr, day1te])
fullday1.head()

,day1
session_id,
1,1.000145
2,1.000145
3,0.999653
4,1.000146
5,1.000145


In [8]:
#Создание фич про утро не утро первой сессии 

In [43]:
#Для train 
time1tr = pd.DataFrame({'time1': train_df['time1']})
time1tr.time1 = time1tr.time1.apply(lambda x: x[11:13]).astype('int').apply(lambda x: 1 if x<11 else 0)

In [44]:
#For TEst
time1te = pd.DataFrame({'time1': test_df['time1']})
time1te.time1 = time1te.time1.apply(lambda x: x[11:13]).astype('int').apply(lambda x: 1 if x<11 else 0)

In [45]:
#Создание единого датафрейма для матрици
time1full = pd.concat([time1tr, time1te])


In [12]:
#Фича про начало месяца 
#for Train 
month1tr = pd.DataFrame({'time1': train_df['time1']})
month1tr.time1 = month1tr.time1.apply(lambda x: x[5:7]) #.astype('int').apply(lambda x: 1 if x<11 else 0)


In [34]:
#for Test 
month1te = pd.DataFrame({'time1': test_df['time1']})
month1te.time1 = month1te.time1.apply(lambda x: x[5:7]) #.astype('int').apply(lambda x: 1 if x<11 else 0)
month1te = month1te.astype(int)

In [14]:
#Создание одного дф по Первому месяцу для первой сессии 
fullmonth1 = pd.concat([month1tr, month1te]).astype(int)

In [15]:
#форма у полного игрика 
y.shape[0]

253561

In [16]:
#Создание единого датафрэйма

In [17]:
full_df = pd.concat([Xtr, Xte])
full_df

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0
3,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0
5,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0
...,...,...,...,...,...,...,...,...,...,...
82793,812,1039.0,676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82794,300,302.0,302.0,300.0,300.0,1222.0,302.0,1218.0,1221.0,1216.0
82795,29,33.0,35.0,22.0,37.0,6779.0,30.0,21.0,23.0,6780.0


In [70]:
all_values_of_full_df = full_df.values.flatten() #все значения в каждом столбце 

In [71]:

# искомая матрица
Xoffulldf = csr_matrix( ([1] * all_values_of_full_df.shape[0],
                                all_values_of_full_df,
                                range(0, all_values_of_full_df.shape[0] + 10, 10))) [:, 1:]
                                

In [72]:
Xoffulldf

<336358x48371 sparse matrix of type '<class 'numpy.intc'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [73]:
#Объединение матриц сайтов и значений первого включения time1 for train and test df + новая фича показывющая в целом весь первый день первой сессии
Xoffulldf = csr_matrix(hstack([Xoffulldf, time1full, fullmonth1]))


In [74]:
Xoffulldf

<336358x48373 sparse matrix of type '<class 'numpy.int64'>'
	with 2321223 stored elements in Compressed Sparse Row format>

In [75]:
#Получили окончательный формат для Х и у над которыми можно колдовать

In [76]:
X = Xoffulldf[:y.shape[0]] #вернули кол-во то что было в тестовой выборке

In [77]:
X

<253561x48373 sparse matrix of type '<class 'numpy.int64'>'
	with 1774364 stored elements in Compressed Sparse Row format>

In [78]:
y = train_df['target']
y

session_id
1         0
2         0
3         0
4         0
5         0
         ..
253557    0
253558    0
253559    0
253560    0
253561    0
Name: target, Length: 253561, dtype: int64

In [79]:
#Деление выборок на тренировку и тест и Разработка Логистической модели 

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=17)

In [81]:
LogistRegr = LogisticRegression(C=1.0, n_jobs=-1)

In [82]:
%%time
LogistRegr.fit(X_train, y_train)

Wall time: 7.04 s


LogisticRegression(n_jobs=-1)

In [83]:
ypred = LogistRegr.predict_proba(X_test)[:,1]
ypred

array([4.00204308e-03, 4.36236482e-03, 4.90490275e-05, ...,
       5.27020828e-04, 1.45858052e-04, 6.68338006e-07])

In [84]:
roc_auc_score(y_test, ypred)

0.9832282961136228

In [197]:
#Досоздание матрица для соревнования на Kaggle 


In [198]:
#Х матрица для ТЕСТовых данных для предсказания 

In [85]:
test_values_of_full_df = Xte.values.flatten()


In [86]:
XoftestDF = csr_matrix( ([1] * test_values_of_full_df.shape[0],
                                test_values_of_full_df,
                                range(0, test_values_of_full_df.shape[0] + 10, 10))) [:, 1:]

XoftestDF = csr_matrix(hstack([XoftestDF,time1te, month1te])) #объединение матрицы + значений  времени для тестовой части + фичи day1 

In [87]:
XoftestDF

<82797x48373 sparse matrix of type '<class 'numpy.int64'>'
	with 546859 stored elements in Compressed Sparse Row format>

In [88]:
yfinalpred = LogistRegr.predict_proba(XoftestDF)[:,1]
yfinalpred

array([9.15097234e-03, 4.90653448e-11, 5.19025118e-11, ...,
       1.52064413e-02, 2.06570447e-05, 2.29313613e-07])

In [89]:
#Создание таблицы для соревнования

pd.DataFrame({'session_id': range(1, yfinalpred.shape[0] + 1), 'target': yfinalpred}).set_index('session_id').to_csv('Try10_Catch_Alice.csv')


d


In [ ]:
edeede